In [5]:
from sqlalchemy import create_engine, Column, Integer, String, Time, ForeignKey
from sqlalchemy.orm import sessionmaker, DeclarativeBase
import pandas as pd

df_samples = pd.read_pickle('../data/samples.pkl').head()
#df_samples

In [6]:
class Base(DeclarativeBase):
    pass

class BandInfo(Base):
    __tablename__ = 'band_info'

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    note = Column(String)
    
    def __init__(self, name, note): 
        self.name = name
        self.note = note

class AlbumInfo(Base):
    __tablename__ = 'album_info'

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    band_id = Column(Integer) # foreign key Band

    def __init__(self, name, band_id): 
        self.name = name
        self.band_id = band_id

class SongInfo(Base):
    __tablename__ = 'song_info'

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    album_id = Column(Integer) # foreign key Album

    def __init__(self, name, album_id): 
        self.name = name
        self.album_id = album_id

class SourceInfo(Base):
    __tablename__ = 'source_info'

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)

    def __init__(self, name): 
        self.name = name

class SampleInfo(Base):
    __tablename__ = 'sample_info'

    id = Column(Integer, primary_key=True, autoincrement=True)
    text = Column(String)
    song_id = Column(Integer) # FK Song
    source_id = Column(Integer) # FK Source
    timestamp_song = Column(Time)
    timestamp_source = Column(Time)
    note = Column(String)

    def __init__(self, text, song_id, source_id, timestamp_song, timestamp_source, note): 
        self.text = text
        self.song_id = song_id
        self.source_id = source_id
        self.timestamp_song = timestamp_song
        self.timestamp_source = timestamp_source
        self.note = note

